In [1]:
%pip install torch
%pip install torchvision
%pip install "numpy<2"
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import time

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import OneCycleLR

# dataset
class BreakHisDataset(Dataset):
    def __init__(self, csv_file, root_dir, train=True, transform=None):
        
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        if train:
            self.data_frame = self.data_frame[self.data_frame['grp'].str.lower() == "train"]
        else:
            self.data_frame = self.data_frame[self.data_frame['grp'].str.lower() == "test"]
        
        self.data_frame.reset_index(drop=True, inplace=True)

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        row = self.data_frame.iloc[idx]
        filename = row['filename']
        
        img_path = os.path.join(self.root_dir, filename)
        image = Image.open(img_path).convert('RGB')
    
        lower_filename = filename.lower()
        if "benign" in lower_filename:
            label = 0
        elif "malignant" in lower_filename:
            label = 1
        else:
            raise ValueError(f"Cannot determine label from filename: {filename}")

        if self.transform:
            image = self.transform(image)
        return image, label

# ViT components
class PatchEmbedding(nn.Module):
    def __init__(self, d_model, img_size, patch_size, n_channels):
        super().__init__()
        self.d_model = d_model
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_channels = n_channels
        self.linear_project = nn.Conv2d(self.n_channels, self.d_model,
                                        kernel_size=self.patch_size, stride=self.patch_size)

    def forward(self, x):
        x = self.linear_project(x)
        x = x.flatten(2)
        x = x.transpose(1, 2)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super().__init__()
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        pe = torch.zeros(max_seq_length, d_model)
        for pos in range(max_seq_length):
            for i in range(d_model):
                if i % 2 == 0:
                    pe[pos][i] = np.sin(pos / (10000 ** (i / d_model)))
                else:
                    pe[pos][i] = np.cos(pos / (10000 ** ((i - 1) / d_model)))
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        tokens_batch = self.cls_token.expand(x.size(0), -1, -1)
        x = torch.cat((tokens_batch, x), dim=1)
        x = x + self.pe
        return x

class AttentionHead(nn.Module):
    def __init__(self, d_model, head_size):
        super().__init__()
        self.head_size = head_size
        self.query = nn.Linear(d_model, head_size)
        self.key = nn.Linear(d_model, head_size)
        self.value = nn.Linear(d_model, head_size)

    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        attention = Q @ K.transpose(-2, -1)
        attention = attention / (self.head_size ** 0.5)
        attention = torch.softmax(attention, dim=-1)
        out = attention @ V
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.head_size = d_model // n_heads
        self.W_o = nn.Linear(d_model, d_model)
        self.heads = nn.ModuleList([AttentionHead(d_model, self.head_size) for _ in range(n_heads)])

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.W_o(out)
        return out

class TransformerEncoder(nn.Module):
    def __init__(self, d_model, n_heads, r_mlp=4):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.mha = MultiHeadAttention(d_model, n_heads)
        self.ln2 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model * r_mlp),
            nn.GELU(),
            nn.Linear(d_model * r_mlp, d_model)
        )

    def forward(self, x):
        out = x + self.mha(self.ln1(x))
        out = out + self.mlp(self.ln2(out))
        return out

class VisionTransformer(nn.Module):
    def __init__(self, d_model, n_classes, img_size, patch_size, n_channels, n_heads, n_layers):
        super().__init__()
        assert img_size[0] % patch_size[0] == 0 and img_size[1] % patch_size[1] == 0, \
            "img_size dimensions must be divisible by patch_size dimensions"
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_classes = n_classes
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_channels = n_channels
        self.n_heads = n_heads
        self.n_patches = (self.img_size[0] * self.img_size[1]) // (self.patch_size[0] * self.patch_size[1])
        self.max_seq_length = self.n_patches + 1

        self.patch_embedding = PatchEmbedding(self.d_model, self.img_size, self.patch_size, self.n_channels)
        self.positional_encoding = PositionalEncoding(self.d_model, self.max_seq_length)
        self.transformer_encoder = nn.Sequential(
            *[TransformerEncoder(self.d_model, self.n_heads) for _ in range(n_layers)]
        )
        self.classifier = nn.Linear(self.d_model, 1)

    def forward(self, images):
        x = self.patch_embedding(images)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = self.classifier(x[:, 0])
        return x

# hyperparameters
d_model = 9
n_classes = 2 # Binary classification: 0 for benign, 1 for malignant
img_size = (256, 256)
patch_size = (16, 16)
n_channels = 3
n_heads = 3
n_layers = 8
batch_size = 512
epochs = 100
alpha = 0.001

transform = T.Compose([
    T.Resize(img_size),
    T.ToTensor()
])

test_transform = T.Compose([
    T.Resize(img_size),
    T.RandomRotation(degrees=180),
    T.ToTensor()
])


csv_file = "Folds.csv"
root_dir = "./../BreaKHis_v1/"

train_set = BreakHisDataset(csv_file=csv_file, root_dir=root_dir, train=True, transform=transform)
test_set = BreakHisDataset(csv_file=csv_file, root_dir=root_dir, train=False, transform=test_transform)

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size, num_workers=8, pin_memory=True)

print("Training set grp values:", train_set.data_frame['grp'].unique())
print("Testing set grp values:", test_set.data_frame['grp'].unique())

cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using device:", device, f"({torch.cuda.get_device_name(device)})")
else:
    print("Using device:", device)

transformer = VisionTransformer(d_model, n_classes, img_size, patch_size, n_channels, n_heads, n_layers).to(device)
optimizer = AdamW(transformer.parameters(), lr=alpha)
scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=epochs)
criterion = nn.BCEWithLogitsLoss()

# Train loop
for epoch in range(epochs):
    start = time.time()
    transformer.train()
    training_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = transformer(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        training_loss += loss.item()
    end = time.time()
    curr_lr = scheduler.get_last_lr()
    print(f'Epoch {epoch + 1}/{epochs} loss: {training_loss / len(train_loader):.3f} time: {end-start:.2f} sec lr: {curr_lr}')
    if ((epoch+1) % 10) == 0:
        torch.save(transformer.state_dict(), f"./checkpoints/checkpoint:{epoch+1}-{d_model}-{n_classes}-{img_size}-{patch_size}-{n_channels}-{n_heads}-{n_layers}-{batch_size}-{epochs}-{alpha}.pth")

# Testing
def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predicted = (torch.sigmoid(outputs) >= 0.5).float()
            labels = labels.float().unsqueeze(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'\nModel Accuracy: {accuracy:.2f}%')
    return accuracy

test_accuracy = test_model(transformer, test_loader, device)

# Save the final model
model_scripted = torch.jit.script(transformer)
model_scripted.save(f"./models/binary/{test_accuracy:.2f}%-{d_model}-{n_classes}-{img_size}-{patch_size}-{n_channels}-{n_heads}-{n_layers}-{batch_size}-{epochs}-{alpha}.pth")

# model = torch.jit.load('model_scripted.pt')
# model.eval()


Training set grp values: ['train']
Testing set grp values: ['test']
Using device: cuda (NVIDIA GeForce RTX 4050 Laptop GPU)
Epoch 1/100 loss: 0.622 time: 154.00 sec lr: [0.0004263292768923316]
Epoch 2/100 loss: 0.581 time: 154.00 sec lr: [0.0005050282613936266]
Epoch 3/100 loss: 0.461 time: 153.42 sec lr: [0.0006352335837730184]
Epoch 4/100 loss: 0.444 time: 157.05 sec lr: [0.0008155168223697683]
Epoch 5/100 loss: 0.433 time: 154.17 sec lr: [0.001043900174138846]
Epoch 6/100 loss: 0.439 time: 153.69 sec lr: [0.00131787815220262]
Epoch 7/100 loss: 0.443 time: 154.36 sec lr: [0.001634445072374903]


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>
Traceback (most recent call last):
  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>
Traceback (most recent call last):
  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/root

Epoch 8/100 loss: 0.426 time: 154.54 sec lr: [0.0019901280271156247]


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>
Traceback (most recent call last):
  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>
Traceback (most recent call last):
  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/root

Epoch 9/100 loss: 0.416 time: 161.72 sec lr: [0.0023810249851742675]


...

Epoch 59/100 loss: 0.017 time: 170.49 sec lr: [0.0063259569828980015]


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>
Exception ignored in: Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__

Traceback (most recent call last):
      File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
self._shutdown_workers()
      File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
self._shutdown_workers()
      File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
if w.is_alive():    
if w.is_alive(): 
            ^ ^^^^^^^^^^^^^^^^^^^^^
^  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
^

Epoch 60/100 loss: 0.019 time: 164.95 sec lr: [0.006108330133495992]


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>Traceback (most recent call last):

  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
Traceback (most recent call last):
      File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
self._shutdown_workers()Exception ignored in:     
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fd60f145b20>self._shutdown_workers()  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers


Exception ignored in:     Traceback (most recent call last):
  File "/root/DS3_Project_2025/testing/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
if w.is_a

Epoch 61/100 loss: 0.163 time: 162.27 sec lr: [0.00588847129498195]
Epoch 62/100 loss: 0.026 time: 172.47 sec lr: [0.005666823233797245]
Epoch 63/100 loss: 0.012 time: 170.05 sec lr: [0.005443832319639601]
Epoch 64/100 loss: 0.008 time: 168.08 sec lr: [0.005219947626533916]
Epoch 65/100 loss: 0.010 time: 172.40 sec lr: [0.004995620028456936]
Epoch 66/100 loss: 0.008 time: 173.74 sec lr: [0.004771301291337086]
Epoch 67/100 loss: 0.019 time: 176.32 sec lr: [0.004547443163257993]
Epoch 68/100 loss: 0.022 time: 170.62 sec lr: [0.0043244964646979855]
Epoch 69/100 loss: 0.006 time: 169.86 sec lr: [0.004102910180637627]
Epoch 70/100 loss: 0.002 time: 171.07 sec lr: [0.0038831305563637323]
Epoch 71/100 loss: 0.001 time: 171.41 sec lr: [0.0036656001987907345]
Epoch 72/100 loss: 0.000 time: 168.50 sec lr: [0.0034507571851092786]
Epoch 73/100 loss: 0.000 time: 166.57 sec lr: [0.0032390341805570424]
Epoch 74/100 loss: 0.000 time: 163.68 sec lr: [0.0030308575670885416]
Epoch 75/100 loss: 0.000 time